# [FMA: A Dataset For Music Analysis](https://github.com/mdeff/fma)

Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

## Baselines

* This notebook evaluates standard classifiers from scikit-learn on the provided features.
* Moreover, it evaluates Deep Learning models on both audio and spectrograms.

In [1]:
import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import keras
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier
from catboost import CatBoostClassifier

import utils

In [2]:
AUDIO_DIR = "../data/fma_medium/"

tracks = utils.load('../data/fma_metadata/tracks.csv')
features = utils.load('../data/fma_metadata/features.csv')
# features_spec = utils.load('../data/fma_metadata/features_spectogram.csv')
echonest = utils.load('../data/fma_metadata/echonest.csv')

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

## Subset

In [3]:
np.where(np.isnan(features_all))

NameError: name 'features_all' is not defined

In [8]:
subset = tracks.index[tracks['set', 'subset'] <= 'medium']

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

# features_all = features.join(features_spec, how='inner').sort_index(axis=1)

tracks = tracks.loc[subset]
features_all = features.loc[subset]

# tracks = tracks[tracks['track_id'] not in np.unique(np.where(np.isnan(features_all))[0])]
# features_all.dropna(axis=0, inplace=True)

print(tracks.shape, features_all.shape)

Not enough Echonest features: (13129, 767)
(25000, 52) (25000, 518)


## Feature Engineering

Lets remove some features that have > 95% correlation, this can be a pain in the butt for some classifiers to overcome

In [4]:
def fset_corr_to_drop(fset):
    corr = features_all[fset].corr()
    cor_matrix = corr.abs()
    upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape),k=1).astype(np.bool))  # Get upper tri so we don't have redundancy
    to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]  # Find any to drop that are > 95%
    
    return to_drop

In [ ]:
from itertools import combinations

feature_list = features_all.columns.levels[0]
features_no_high_corr = {}

# Note this takes a crap ton of time to calculate, time for some coffee or something;
# You know, I have a thing against people who like really really dark coffee, like why did *someone* get me this really really really dark coffee
# Like dark coffee has lower caffeine ratio cuz you burn off all the good stuff, and the taste is akin to charcoal
# This is like eating the charcoal off a grill instead of the sweet juicy meat, mmmm charcoal, who says that?
# At least charcoal has the saving grace of the traditional woody smell that makes men men, but dark coffee? the smell just makes it worse
# Its like, I used to have some span of nose hair before, but now its all gone... just gone I tell you
for i in range(1, len(feature_list) + 1):  # The higher the range, the longer it takes, don't get your hopes up of how fast this runs initially
    print(f'Calculating combinations of range({i})')
    for fset in list(combinations(feature_list, i)):  # get a list of all possible combinations and find the features to drop for each set!
        print(f'Calculating correlation for {fset}')
        result = fset_corr_to_drop(list(fset))
        features_no_high_corr[fset] = result

In [94]:
import pickle

pickle.dump(features_no_high_corr, open('features_no_high_corr_abs.pickle', 'wb'))  # Well... we can't do that thing everyy single time, this is way faster

In [ ]:
import pickle

features_no_high_corr = pickle.load(open('features_no_high_corr.pickle', 'wb'))

In [70]:
for key, value in features_no_high_corr.items():
    print(key, value)

('chroma_cens',) [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12')]
('chroma_cqt',) [('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12')]
('chroma_stft',) [('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'), ('chroma_stft', 'kurtosis', '07'), ('chroma_stft', 'kurtosis', '11'), ('chroma_stft', 'kurtosis',

('chroma_stft', 'spectral_bandwidth') [('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'), ('chroma_stft', 'kurtosis', '07'), ('chroma_stft', 'kurtosis', '11'), ('chroma_stft', 'kurtosis', '12'), ('chroma_stft', 'median', '01'), ('chroma_stft', 'median', '02'), ('chroma_stft', 'median', '03'), ('chroma_stft', 'median', '04'), ('chroma_stft', 'median', '05'), ('chroma_stft', 'median', '06'), ('chroma_stft', 'median', '07'), ('chroma_stft', 'median', '08'), ('chroma_stft', 'median', '09'), ('chroma_stft', 'median', '10'), ('chroma_stft', 'median', '11'), ('chroma_stft', 'median', '12'), ('spectral_bandwidth', 'median', '01')]
('chroma_stft', 'spectral_centroid') [('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'), ('chroma_stft', 'kurtosis', '07'), ('chroma_stft', 'kurtosis', '11'), ('chroma_stft', 'kurtosis', '12'), ('chroma_stft', 'median', '01'), ('chroma_stft', 'median', '02'), ('chroma_stft', 'median', '03'), ('chroma_stft', 'median', '04'), ('chr

('chroma_cens', 'chroma_cqt', 'rmse') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12'), ('rmse', 'median', '01')]
('chroma_cens', 'chroma_cqt', 'spectral_bandwidth') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('c

('chroma_cens', 'mfcc', 'rmse') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('mfcc', 'median', '01'), ('mfcc', 'median', '02'), ('mfcc', 'median', '03'), ('mfcc', 'median', '04'), ('mfcc', 'median', '05'), ('mfcc', 'median', '06'), ('mfcc', 'median', '07'), ('mfcc', 'median', '08'), ('mfcc', 'median', '09'), ('mfcc', 'median', '10'), ('mfcc', 'median', '11'), ('mfcc', 'median', '12'), ('mfcc', 'median', '13'), ('mfcc', 'median', '14'), ('mfcc', 'median', '15'), ('mfcc', 'median', '16'), ('mfcc', 'median', '17'), ('mfcc', 'median', '18'), ('mfcc', 'median', '19'), ('mfcc', 'median', '20'), ('rmse', 'median', '01')]
('chroma_cens', 'mfcc', '

('chroma_cens', 'spectral_centroid', 'zcr') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('spectral_centroid', 'median', '01')]
('chroma_cens', 'spectral_contrast', 'spectral_rolloff') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('spectral_contrast', 'median', '01'), ('spectral_contrast', 'medi

...

('chroma_cens', 'chroma_cqt', 'rmse', 'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast', 'spectral_rolloff', 'tonnetz', 'zcr') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12'), ('rmse', 'median', '01'), ('spectral_bandwidth', 'median', '01'), ('spectral_ce

('chroma_cens', 'chroma_stft', 'mfcc', 'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast', 'spectral_rolloff', 'tonnetz', 'zcr') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'), ('chroma_stft', 'kurtosis', '07'), ('chroma_stft', 'kurtosis', '11'), ('chroma_stft', 'kurtosis', '12'), ('chroma_stft', 'median', '01'), ('chroma_stft', 'median', '02'), ('chroma_stft', 'median', '03'), ('chroma_stft', 'median', '04'), ('chroma_stft', 'median', '05'), ('chroma_stft', 'median', '06'), ('chroma_stft', 'median', '07'), ('chroma_stft', 'median', '08'), ('chroma_stft', 'median

('chroma_cqt', 'chroma_stft', 'mfcc', 'rmse', 'spectral_bandwidth', 'spectral_contrast', 'spectral_rolloff', 'tonnetz', 'zcr') [('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12'), ('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'), ('chroma_stft', 'kurtosis', '07'), ('chroma_stft', 'kurtosis', '11'), ('chroma_stft', 'kurtosis', '12'), ('chroma_stft', 'median', '01'), ('chroma_stft', 'median', '02'), ('chroma_stft', 'median', '03'), ('chroma_stft', 'median', '04'), ('chroma_stft', 'median', '05'), ('chroma_stft', 'median', '06'), ('chroma_stft', 'median', '07'), ('chroma_stft', 'median', '08'), ('chroma_stft', 'median', '09'), ('chroma_stft', 

('chroma_cens', 'chroma_cqt', 'chroma_stft', 'mfcc', 'rmse', 'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast', 'spectral_rolloff', 'zcr') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12'), ('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '05'

('chroma_cens', 'chroma_cqt', 'chroma_stft', 'rmse', 'spectral_bandwidth', 'spectral_centroid', 'spectral_contrast', 'spectral_rolloff', 'tonnetz', 'zcr') [('chroma_cens', 'median', '01'), ('chroma_cens', 'median', '02'), ('chroma_cens', 'median', '03'), ('chroma_cens', 'median', '04'), ('chroma_cens', 'median', '05'), ('chroma_cens', 'median', '06'), ('chroma_cens', 'median', '07'), ('chroma_cens', 'median', '08'), ('chroma_cens', 'median', '09'), ('chroma_cens', 'median', '10'), ('chroma_cens', 'median', '11'), ('chroma_cens', 'median', '12'), ('chroma_cqt', 'median', '01'), ('chroma_cqt', 'median', '02'), ('chroma_cqt', 'median', '03'), ('chroma_cqt', 'median', '04'), ('chroma_cqt', 'median', '05'), ('chroma_cqt', 'median', '06'), ('chroma_cqt', 'median', '07'), ('chroma_cqt', 'median', '08'), ('chroma_cqt', 'median', '09'), ('chroma_cqt', 'median', '10'), ('chroma_cqt', 'median', '11'), ('chroma_cqt', 'median', '12'), ('chroma_stft', 'kurtosis', '04'), ('chroma_stft', 'kurtosis', '

## TrainTest Split

In [9]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

19922 training examples, 2505 validation examples, 2573 testing examples
Top genres (16): ['Blues', 'Classical', 'Country', 'Easy Listening', 'Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Jazz', 'Old-Time / Historic', 'Pop', 'Rock', 'Soul-RnB', 'Spoken']
All genres (151): [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 37, 38, 41, 42, 43, 45, 46, 47, 49, 53, 58, 63, 64, 65, 66, 70, 71, 74, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 97, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 137, 138, 166, 167, 169, 171, 172, 174, 177, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 311, 314, 322, 337, 359, 360, 361, 362, 374, 378, 400, 401, 404, 428, 439, 440, 441, 442, 443, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 651, 659, 695, 741, 763, 808, 810, 811, 906, 1032, 1060, 1193, 1235]


## 1 Multiple classifiers and feature sets

Todo:
* Cross-validation for hyper-parameters.
* Dimensionality reduction?

### 1.1 Pre-processing

In [10]:
def find_features_to_drop(columns):
    keys = [set(k) for k in features_no_high_corr.keys()]  # Convert keys into sets (cant use sets as keys)
    idx = [set(columns) == k for k in keys].index(True)    # Find the set that equals our target (need to check sets for out-of-order equivalance)
    return features_no_high_corr[list(features_no_high_corr.keys())[idx]]

ol = {}



def find_features_to_drop_rt(columns):
    keys = [set(k) for k in ol.keys()]  # Convert keys into sets (cant use sets as keys)
    if(set(columns) in keys):
        return ol[list(ol.keys())[[set(columns) == k for k in keys].index(True)]]
    else:
        td = fset_corr_to_drop(columns)
        ol[tuple(columns)] = td
        return td

In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.cluster import KMeans

def pre_process(tracks, features, columns, multi_label=False, verbose=False):
    if not multi_label:
        # Assign an integer value to each genre.
        enc = LabelEncoder()
        labels = tracks['track', 'genre_top']
        #y = enc.fit_transform(tracks['track', 'genre_top'])
    else:
        # Create an indicator matrix.
        enc = MultiLabelBinarizer()
        labels = tracks['track', 'genres_all']
        #labels = tracks['track', 'genres']

    # Split in training, validation and testing sets.
    y_train = enc.fit_transform(labels[train])
    y_val = enc.transform(labels[val])
    y_test = enc.transform(labels[test])
    
    X_train = features.loc[train, columns]
    X_val = features.loc[val, columns]
    X_test = features.loc[test, columns]
    
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    
    ## Try KMeans
    # clf = KMeans(n_clusters=len(np.unique(y_train)), random_state=42)
    # clf.fit(X_train)
    # kmeans_train = clf.labels_
    # kmeans_test = clf.predict(X_test)
    
    # X_train[['km {}'.format(k) for k in range(16)]] = pd.get_dummies(kmeans_train)
    # X_test[['km {}'.format(k) for k in range(16)]] = pd.get_dummies(kmeans_test)
    
    X_train = X_train.values
    X_test = X_test.values
    X_val = X_val.values
    
    # Standardize features by removing the mean and scaling to unit variance.
    # scaler = MinMaxScaler(copy=False)
    scaler = StandardScaler(copy=False)
    scaler.fit_transform(X_train)
    # scaler.transform(X_val)
    scaler.transform(X_test)
    
    # imp = SimpleImputer(missing_values=np.nan, strategy="mean")
    # imp.fit_transform(X_train)
    # imp.transform(X_val)
    # imp.transform(X_test)
    
    return y_train, y_val, y_test, X_train, X_val, X_test

### 1.2 Single genre

In [12]:
from sklearn.feature_selection import SelectKBest, chi2

def test_classifiers_features(classifiers, feature_sets, multi_label=False):
    columns = list(classifiers.keys()).insert(0, 'dim')
    scores = pd.DataFrame(columns=columns, index=feature_sets.keys())
    times = pd.DataFrame(columns=classifiers.keys(), index=feature_sets.keys())
    for fset_name, fset in tqdm_notebook(feature_sets.items(), desc='features'):
        y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features_all, fset, multi_label)
        # y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features.drop(find_features_to_drop(fset), axis=1), fset, multi_label)
        # y_train, y_val, y_test, X_train, X_val, X_test = pre_process(tracks, features.drop(find_features_to_drop_rt(fset), axis=1), fset, multi_label)
        
        for k in [-1]: #, X_train.shape[1] *  3 // 4, X_train.shape[1] // 2, X_train.shape[1] // 4]:
            if(k != -1):
                selectors = SelectKBest(chi2, k=k)
                X_train_n = selectors.fit_transform(X_train, y_train)
                X_test_n = selectors.transform(X_test)

            scores.loc[fset_name, 'dim'] = X_train.shape[1]
            for clf_name, clf in classifiers.items():  # tqdm_notebook(classifiers.items(), desc='classifiers', leave=False):
                t = time.process_time()
                clf.fit(X_train if k == -1 else X_train_n, y_train)
                score = clf.score(X_test if k == - 1 else X_test_n, y_test)
                scores.loc[fset_name, clf_name + "k{}".format(k)] = score
                times.loc[fset_name, clf_name + "k{}".format(k)] = time.process_time() - t
    return scores, times

def format_scores(scores):
    def highlight(s):
        is_max = s == max(s[1:])
        return ['background-color: yellow' if v else '' for v in is_max]
    scores = scores.style.apply(highlight, axis=1)
    return scores.format('{:.2%}', subset=pd.IndexSlice[:, scores.columns[1]:])

In [ ]:
classifiers = {
    # 'LR': LogisticRegression(),
    # 'kNN': KNeighborsClassifier(n_neighbors=200),
    # 'SVCrbfC1': SVC(kernel='rbf', C=1),
    # 'SVCrbfC10': SVC(kernel='rbf', C=10),
    # 'SVCrbfC0.1': SVC(kernel='rbf', C=0.1),
    # "CATBoost": CatBoostClassifier(iterations=1000, silent=True),
    # 'SVCpolyD1C1': SVC(kernel='poly', degree=1, C=1),
    # 'SVCpolyD1C10': SVC(kernel='poly', degree=1, C=10),
    # 'SVCpolyD1C0.1': SVC(kernel='poly', degree=1, C=0.1),
    # 'SVCpolyD2C1': SVC(kernel='poly', degree=2, C=1),
    # 'SVCpolyD2C10': SVC(kernel='poly', degree=2, C=10),
    # 'SVCpolyD2C0.1': SVC(kernel='poly', degree=2, C=0.1),
    #  'SVCpolyD4C1': SVC(kernel='poly', degree=4, C=1),
     # 'SVCpolyD4C10': SVC(kernel='poly', degree=4, C=10),
     # 'SVCpolyD4C0.1': SVC(kernel='poly', degree=4, C=0.1),
    # 'linSVC1': SVC(kernel="linear"),
    # 'linSVC2': LinearSVC(),
    # GaussianProcessClassifier(1.0 * RBF(1.0), warm_start=True),
    # 'DT': DecisionTreeClassifier(max_depth=5),
    # 'RFDefaultGini': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    # 'RFDefaultEntropy': RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1, criterion='entropy'),
    # 'RFDefaultGiniMFAuto': RandomForestClassifier(max_depth=5, n_estimators=10, max_features='auto'),
    # 'RFDefaultEntropyMFAuto': RandomForestClassifier(max_depth=5, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=10NE=10': RandomForestClassifier(max_depth=10, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=20NE=10': RandomForestClassifier(max_depth=20, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=35NE=10': RandomForestClassifier(max_depth=35, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=10': RandomForestClassifier(max_depth=50, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=5NE=10': RandomForestClassifier(max_depth=5, n_estimators=10, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=10NE=25': RandomForestClassifier(max_depth=10, n_estimators=25, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=20NE=50': RandomForestClassifier(max_depth=20, n_estimators=50, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=35NE=75': RandomForestClassifier(max_depth=35, n_estimators=75, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=100': RandomForestClassifier(max_depth=50, n_estimators=100, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=65NE=100': RandomForestClassifier(max_depth=65, n_estimators=100, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=80NE=100': RandomForestClassifier(max_depth=80, n_estimators=100, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=125': RandomForestClassifier(max_depth=50, n_estimators=125, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=150': RandomForestClassifier(max_depth=50, n_estimators=150, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=125': RandomForestClassifier(max_depth=65, n_estimators=125, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=125': RandomForestClassifier(max_depth=65, n_estimators=150, max_features='auto', criterion='entropy'),
    # 'RFEntropyMD=50NE=125': RandomForestClassifier(max_depth=75, n_estimators=175, max_features='auto', criterion='entropy'),
    
    
    # 'RFGiniMD=50NE=75': RandomForestClassifier(max_depth=50, n_estimators=75, max_features=1),
    # 'RFGiniMD=75NE=75': RandomForestClassifier(max_depth=75, n_estimators=75, max_features=1),
    # 'AdaBoost': AdaBoostClassifier(n_estimators=500, learning_rate=0.01),
    'MLP1': MLPClassifier(hidden_layer_sizes=(512, 256, 384, 256, 512, 384, 384, 256, 128, 32, 16), max_iter=1000, learning_rate='adaptive', ),
    # 'MLP2': MLPClassifier(hidden_layer_sizes=(200, 50), max_iter=2000),
    # 'NB': GaussianNB(),
    # 'QDA': QuadraticDiscriminantAnalysis(),
}

feature_sets = {
#   'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_social': ('echonest', 'social_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
#    'echonest_audio/social': ('echonest', ('audio_features', 'social_features')),
#    'echonest_all': ('echonest', ('audio_features', 'social_features', 'temporal_features')),
}
# for name in features.columns.levels[0]:
#     feature_sets[name] = name
feature_sets.update({
    # 'spectogram': ['spectogram'],
    #'mfcc': ['mfcc']
    #'contrast': ['spectral_contrast'],
    #'contrast/chroma': ['spectral_contrast', 'chroma_cens'],
    #'contrast/centroid': ['spectral_contrast', 'spectral_centroid'],
    #'contrast/chroma/centroid': ['spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    # 'mfcc': ['mfcc'],
    # 'mfcc/chroma': ['mfcc', 'chroma_cens'],
    # 'mfcc/centroid': ['mfcc', 'spectral_centroid'],
    # 'mfcc/contrast/centroid': ['mfcc', 'spectral_contrast', 'spectral_centroid'],
    # 'mfcc/contrast/centroid/bandwidth': ['mfcc', 'spectral_contrast', 'spectral_centroid', 'bandwidth'],
    # 'mfcc&contrast/chroma/centroid': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid'],
    'mfcc&contrast/chroma/centroid/tonnetz/bandwidth': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz', 'spectral_bandwidth'],
    # 'mfcc&contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    # 'mfcc&contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
    # 'mfcc&contrast/chroma/centroid/bandwidth': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'spectral_bandwidth'],
    # 'all_non-echonest': list(features.columns.levels[0])
})

scores, times = test_classifiers_features(classifiers, feature_sets)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

In [22]:
from matplotlib import pyplot as plt

loss_values = classifiers["MLP1"].loss_curve_
plt.plot(loss_values)

KeyError: 'MLP1'

### 1.3 Multiple genres

Todo:
* Ignore rare genres? Count them higher up in the genre tree? On the other hand it's not much tracks.

In [24]:
classifiers = {
    #LogisticRegression(),
    # 'LR': OneVsRestClassifier(LogisticRegression()),
    # 'SVC': OneVsRestClassifier(SVC()),
    # 'MLP': MLPClassifier(max_iter=700),
    "CATBoost": CatBoostClassifier(iterations=1000, silent=True),
}

feature_sets = {
#    'echonest_audio': ('echonest', 'audio_features'),
#    'echonest_temporal': ('echonest', 'temporal_features'),
    'mfcc': 'mfcc',
    # 'mfcc/contrast/chroma/centroid/tonnetz': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'tonnetz'],
    # 'mfcc/contrast/chroma/centroid/zcr': ['mfcc', 'spectral_contrast', 'chroma_cens', 'spectral_centroid', 'zcr'],
}

scores, times = test_classifiers_features(classifiers, feature_sets, multi_label=False)

ipd.display(format_scores(scores))
ipd.display(times.style.format('{:.4f}'))

,dim,CATBoostk-1
mfcc,140.000000,60.86%


,CATBoost,CATBoostk-1
mfcc,nan,3779.8438
